In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pii-dd-mistral-generated/mixtral-8x7b-v1.json
/kaggle/input/pii-detection-removal-from-educational-data/sample_submission.csv
/kaggle/input/pii-detection-removal-from-educational-data/train.json
/kaggle/input/pii-detection-removal-from-educational-data/test.json


In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd 
import os 
import json
import numpy as np
import torch 
import random 


import torch
from torch import nn
import numpy as np
import pandas as pd
from spacy.lang.en import English
from transformers.tokenization_utils import PreTrainedTokenizerBase
from transformers.models.deberta_v2 import DebertaV2ForTokenClassification, DebertaV2TokenizerFast
from transformers.trainer import Trainer
from transformers.training_args import TrainingArguments
from transformers.trainer_utils import EvalPrediction
from transformers.data.data_collator import DataCollatorForTokenClassification
from datasets import Dataset, DatasetDict, concatenate_datasets

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    
SEED = 23
set_seed(SEED)

2024-04-04 06:53:40.414923: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-04 06:53:40.415154: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-04 06:53:40.598387: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
! pip install seqeval

In [4]:
class Config:
    base_save_dir = '/kaggle/working'
    assert os.path.exists(base_save_dir)

    data_dir = '/kaggle/input/pii-detection-removal-from-educational-data/'
    extra_data_dir = '/kaggle/input/pii-dd-mistral-generated/'
    train_file_name = 'train.json'
    extra_file_name = 'mixtral-8x7b-v1.json'
    test_file_name = 'test.json'
    submission_name = 'sample_submission.json'
    TEST_SAMPLES  = 500
    TRAINING_MODEL_PATH = "microsoft/deberta-v3-large"
    TRAINING_MAX_LENGTH = 3500
    EVAL_MAX_LENGTH = 3500
    FREEZE_EMBEDDINGS = True
    FREEZE_LAYERS = 20

In [5]:
cfg = Config()

# EDA and Data Pre-Processing

In [6]:
train_data = json.load(open(os.path.join(cfg.data_dir, cfg.train_file_name)))
test_data = json.load(open(os.path.join(cfg.data_dir, cfg.test_file_name)))
extra_data = json.load(open(os.path.join(cfg.extra_data_dir, cfg.extra_file_name)))

In [7]:
train_data = pd.DataFrame(train_data)
test_data = pd.DataFrame(test_data)
extra_data = pd.DataFrame(extra_data)

train_data.head()

,document,full_text,tokens,trailing_whitespace,labels
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."


In [8]:
extra_data.head()

,document,full_text,tokens,labels,trailing_whitespace
0,dtduupvzgt,"Tiburce Evans, https://www.instagram.com/tibur...","[Tiburce, Evans, ,, https://www.instagram.com/...","[B-NAME_STUDENT, I-NAME_STUDENT, O, B-URL_PERS...","[True, False, True, False, True, True, True, F..."
1,uejmzisyyh,Rose-Mai Rodriguez | PIN # 3814374\n501 Andrea...,"[Rose, -, Mai, Rodriguez, |, PIN, #, 3814374, ...","[B-NAME_STUDENT, I-NAME_STUDENT, I-NAME_STUDEN...","[False, False, True, True, True, True, True, F..."
2,jzevfvijlw,"Lina Wingate; 095 Terry Via\nTerrymouth, MT 24...","[Lina, Wingate, ;, 095, Terry, Via, \n, Terrym...","[B-NAME_STUDENT, I-NAME_STUDENT, O, B-STREET_A...","[True, False, True, True, True, False, False, ..."
3,jqldfongxi,"Lally Scales | Pin # 74,7,9,0, 658-527-2399; l...","[Lally, Scales, |, Pin, #, 74,7,9,0, ,, 658, -...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, B-ID...","[True, True, True, True, True, False, True, Fa..."
4,qzgtouvmtu,"Li Warfield-Fontana; 001-528-777-2370x08989, p...","[Li, Warfield, -, Fontana, ;, 001, -, 528, -, ...","[B-NAME_STUDENT, I-NAME_STUDENT, I-NAME_STUDEN...","[True, False, False, False, True, False, False..."


In [9]:
# Merge train and extra data 
final_data = pd.concat([train_data, extra_data])

# Update document Id as integer
final_data['document'] = [ind+1 for ind in range(len(final_data))]

final_data = final_data.reset_index()

In [10]:
print(f"Final Training Data Len : {final_data.shape[0]}")

Final Training Data Len : 9162


In [11]:
test_data.head()

,document,full_text,tokens,trailing_whitespace
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal..."


In [12]:
print(f"Train Data Len : {final_data.shape[0]}")
print(f"Test Data Len : {test_data.shape[0]}")

Train Data Len : 9162
Test Data Len : 10


In [13]:
train_data.iloc[0]

document                                                               7
full_text              Design Thinking for innovation reflexion-Avril...
tokens                 [Design, Thinking, for, innovation, reflexion,...
trailing_whitespace    [True, True, True, True, False, False, True, F...
labels                 [O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-...
Name: 0, dtype: object

In [14]:
train_data.iloc[0].full_text

"Design Thinking for innovation reflexion-Avril 2021-Nathalie Sylla\n\nChallenge & selection\n\nThe tool I use to help all stakeholders finding their way through the complexity of a project is the  mind map.\n\nWhat exactly is a mind map? According to the definition of Buzan T. and Buzan B. (1999, Dessine-moi  l'intelligence. Paris: Les Éditions d'Organisation.), the mind map (or heuristic diagram) is a graphic  representation technique that follows the natural functioning of the mind and allows the brain's  potential to be released. Cf Annex1\n\nThis tool has many advantages:\n\n•  It is accessible to all and does not require significant material investment and can be done  quickly\n\n•  It is scalable\n\n•  It allows categorization and linking of information\n\n•  It can be applied to any type of situation: notetaking, problem solving, analysis, creation of  new ideas\n\n•  It is suitable for all people and is easy to learn\n\n•  It is fun and encourages exchanges\n\n•  It makes visi

In [15]:
train_data.iloc[0].tokens[7], train_data.iloc[0].trailing_whitespace[7], train_data.iloc[0].labels[7]

('2021', False, 'O')

In [16]:
# Check if token list len and label len are same 
for idx, row in final_data.iterrows():
    assert len(row['tokens']) == len(row['labels']), f"Length of Tokens and Labels do not match"

In [17]:
def check_blank_space_PIL(tokens, white_space, labels):
    """
    Check labels for tokens not separated by blank space
    """
    tokens = np.array(tokens)
    white_space = np.array(white_space)
    labels = np.array(labels)
    
    white_space_indices = np.where(white_space == False)
    PIL_indices = np.where(labels[np.where(white_space[white_space_indices] == False)] != 'O')
    PIL_tokens = tokens[PIL_indices]
    PIL_labels = labels[PIL_indices]
    
    return PIL_indices, PIL_tokens, PIL_labels
        

In [18]:
index = 0
PIL_indices, PIL_tokens, PIL_labels = check_blank_space_PIL(tokens=final_data.iloc[index].tokens,
                                                white_space=final_data.iloc[index].trailing_whitespace,
                                                labels=final_data.iloc[index].labels)
PIL_indices, PIL_tokens, PIL_labels

((array([ 9, 10]),),
 array(['Nathalie', 'Sylla'], dtype='<U16'),
 array(['B-NAME_STUDENT', 'I-NAME_STUDENT'], dtype='<U14'))

In [19]:
index = 10
PIL_indices, PIL_tokens, PIL_labels = check_blank_space_PIL(tokens=final_data.iloc[index].tokens,
                                                white_space=final_data.iloc[index].trailing_whitespace,
                                                labels=final_data.iloc[index].labels)
PIL_indices, PIL_tokens, PIL_labels

((array([30]),),
 array(['Al'], dtype='<U83'),
 array(['B-NAME_STUDENT'], dtype='<U14'))

In [20]:
def get_no_label_ids(data):
    document_ids = []
    for idx, row in data.iterrows():
        label_list = row['labels']
        
        if ['O'] * len(label_list) == label_list:
            document_ids.append(row['document'])
    return document_ids

In [21]:
no_label_doc_ids = get_no_label_ids(data = final_data)
with_label_doc_ids = final_data[~final_data.document.isin(no_label_doc_ids)].document.tolist()
print(f"Document with no PIL Labels : {len(no_label_doc_ids)}\nDocument with PIL Labels: {len(with_label_doc_ids)}")

Document with no PIL Labels : 5862
Document with PIL Labels: 3300


In [22]:
def sample_test_df(df, num_samples):
    test_df = df.sample(n=num_samples)
    train_df = df[~df.document.isin(test_df.document)]
    
    test_df = test_df.reset_index()
    train_df = train_df.reset_index()
    return train_df, test_df


def split_dataset(data, no_label_ids, with_label_ids, test_samples):
    data_no_labels = data[data.document.isin(no_label_ids)]
    data_with_labels = data[data.document.isin(with_label_ids)]
    
    # TEST_SAMPLES // 2 from data_no_labels
    # TEST_SAMPLES // 2 from data_with_labels
    no_label_train_df , no_label_test_df = sample_test_df(df=data_no_labels, 
                                                          num_samples=test_samples//2)
    with_label_train_df, with_label_test_df = sample_test_df(df=data_with_labels, 
                                                          num_samples=test_samples//2)
    
    train_df = pd.concat([with_label_train_df, no_label_train_df])
    test_df = pd.concat([with_label_test_df, no_label_test_df])
    
    #train_df = train_df.reset_index()
    #test_df = test_df.reset_index()
    
    return train_df, test_df

In [23]:
train_df , eval_df = split_dataset(data=final_data,
                                   no_label_ids=no_label_doc_ids,
                                   with_label_ids=with_label_doc_ids,
                                   test_samples=cfg.TEST_SAMPLES
                                  )
print(f"Train Data Size : {train_df.shape[0]}")
print(f"Eval Data Size : {eval_df.shape[0]}")

Train Data Size : 8662
Eval Data Size : 500


# Data Preparation

In [24]:
all_labels = [
    'B-EMAIL', 'B-ID_NUM', 'B-NAME_STUDENT', 'B-PHONE_NUM', 'B-STREET_ADDRESS', 'B-URL_PERSONAL', 'B-USERNAME', 'I-ID_NUM', 'I-NAME_STUDENT', 'I-PHONE_NUM', 'I-STREET_ADDRESS', 'I-URL_PERSONAL', 'O'
]
id2label = {i: l for i, l in enumerate(all_labels)}
label2id = {v: k for k, v in id2label.items()}
target = [l for l in all_labels if l != "O"]

In [25]:
def tokenize(example, tokenizer, label2id):
    # Preprocess the tokens and labels by adding trailing whitespace and labels
    tokens = []
    labels = []
    for token, label, t_ws in zip(example["tokens"], 
                                  example["labels"],
                                  example["trailing_whitespace"]):
        tokens.append(token)
        labels.extend([label] * len(token))
        # Added trailing whitespace and label if true and 
        if t_ws:
            tokens.append(" ")
            labels.append("O")   
    text = "".join(tokens)
    # print(f"len(text)={len(text)}, len(tokens)={len(tokens)}")
    # tokenization without truncation
    tokenized = tokenizer(text, return_offsets_mapping=True,
                          truncation=True, max_length=cfg.TRAINING_MAX_LENGTH)
    labels = np.array(labels)
    
    # Labels
    token_labels = []
    for start_idx, end_idx in tokenized.offset_mapping:
        # Added 'O' 
        if start_idx == 0 and end_idx == 0:
            token_labels.append(label2id["O"])
            continue
        
        # case when the text starts with whitespace
        if text[start_idx].isspace():
            start_idx += 1
        # Convert label to id (int)
        if start_idx< len(labels):
            label_id = label2id[labels[start_idx]]
            token_labels.append(label_id)
    return {**tokenized, "labels": token_labels, "length": len(tokenized.input_ids)}

In [26]:
tokenizer = DebertaV2TokenizerFast.from_pretrained(cfg.TRAINING_MODEL_PATH)
# 'full_text', 'trailing_whitespace'
required_keys = ['tokens', 'trailing_whitespace','labels']
train_dict = DatasetDict({key: train_df[key].tolist() for key in required_keys})
eval_dict = DatasetDict({key: eval_df[key].tolist() for key in required_keys})
train_ds = Dataset.from_dict(train_dict)
eval_ds = Dataset.from_dict(eval_dict)

#num_proc=os.cpu_count()

max_length = cfg.TRAINING_MAX_LENGTH
tokenized_ds = eval_ds.map(tokenize, 
                              fn_kwargs={"tokenizer": tokenizer, 
                                         "label2id": label2id},
                              num_proc=os.cpu_count())

#0:   0%|          | 0/125 [00:00<?, ?ex/s]

#1:   0%|          | 0/125 [00:00<?, ?ex/s]

#2:   0%|          | 0/125 [00:00<?, ?ex/s]

#3:   0%|          | 0/125 [00:00<?, ?ex/s]

In [27]:
from seqeval.metrics import recall_score, precision_score, f1_score
# Compute the model performance metrics using `seqeval`
def compute_metrics(preds):    
    try:
        print("Compute metrics")
        predictions, labels = preds
        predictions = np.argmax(predictions, axis=2)

        # Include prediction Remove ignored index (special tokens)
        true_preds = []
        true_labels = []
        for pred, label in zip(predictions, labels):
            print(f"label = {label}, pred = {pred}")
            true_preds.append([id2label[p] for p, l in zip(pred, label) if l != -100])
            true_labels.append([id2label[l] for p, l in zip(pred, label) if l != -100])
        # Compute recall, precision and f1 score
        recall = recall_score(true_labels, true_preds)
        precision = precision_score(true_labels, true_preds)
        # Use modified f1 score to measure the performance
        f1_score = (1 + 5*5) * recall * precision / (5*5*precision + recall)
        result = {'f1': f1_score,  
                  'recall': recall,
                  'precision': precision}
        print(f"result = {result}")
        del predictions, labels, true_preds, true_labels, preds
        return result
    except Exception as e:
        print(e)

In [28]:
model = DebertaV2ForTokenClassification.from_pretrained(cfg.TRAINING_MODEL_PATH, 
                                                        num_labels=len(all_labels), 
                                                        id2label=id2label, 
                                                        label2id=label2id,
                                                        ignore_mismatched_sizes=True
                                                       )

Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
data_collator=DataCollatorForTokenClassification(tokenizer, pad_to_multiple_of=16)

In [30]:
for param in model.deberta.embeddings.parameters():
    param.requires_grad = cfg.FREEZE_EMBEDDINGS
    
for layer in model.deberta.encoder.layer[:cfg.FREEZE_LAYERS]:
    for param in layer.parameters():
        param.requires_grad = True

In [31]:
#eval_ds_processed.remove_columns(eval_ds.column_names)

In [32]:
import os

train_args = TrainingArguments(
    'test',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    report_to="none"
)

trainer = Trainer(model=model, 
                  args=train_args, 
                  data_collator=data_collator,
                  train_dataset=tokenized_ds,
                  eval_dataset=tokenized_ds,
                  tokenizer=tokenizer,
                  compute_metrics=compute_metrics,
                 )

In [ ]:
# 66888c781d7f4d621566b993017b13cfd65f22c4
trainer.train()